In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.base import Int
from aiida.work.workfunction import workfunction

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist
import ase.build

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
import nglview
from copy import deepcopy

from apps.surfaces.structure_browser import StructureBrowser
from apps.mcfitting.geooptwork import FTGeoOptWorkChain

# Molecule Transformations

In [ ]:
def on_struct_change(c):
    global orig_structure
    orig_structure = None # disable event processing
    s = struct_browser.results.value
    if s:
        orig_structure = s

    update_view()
    
def update_view():
    with info_out:
        clear_output()
        atoms = orig_structure.get_ase()
        
        # remove old components
        if hasattr(viewer, "component_0"):
            viewer.component_0.remove_ball_and_stick()
            viewer.component_0.remove_unitcell()
            cid = viewer.component_0.id
            viewer.remove_component(cid)

        if orig_structure:
            # add new component
            viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
            viewer.add_unitcell()
            viewer.center()

## Step 1: Select structure

In [ ]:
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
display(struct_browser)

In [ ]:
viewer = nglview.NGLWidget()
info_out = ipw.Output()
display(viewer, info_out)

In [ ]:
def dancing_molecule(atoms, phi, y, z):
    slab = atoms[-1568:]
    mol = atoms[:-1568]
    mol.euler_rotate(phi=phi,theta=0,psi=0,center='COP')
    mol.translate((0, y, z))
    atoms = mol+slab
    return atoms


In [ ]:
def molecule_circus():
    atoms = struct_browser.results.value.get_ase()
    
    y_tot = np.sqrt(1.5)*4.206
    phis = [0, 10, 20]
    y_trans = [0, y_tot/4., y_tot/2., 3*y_tot/4.]
    z_trans = [-0.5, 0., 1., 3., 6.]
    
    all_the_structures = dict()
    for z in z_trans:
        for y in y_trans:
            for phi in phis:
                tmp_atoms = deepcopy(atoms)
                new_atoms = dancing_molecule(tmp_atoms, phi, y, z)
                all_the_structures['Phi{} Y{} Z{}'.format(phi, round(y), z)] = new_atoms
    
    return all_the_structures

## Step 2: Transform and store (this will take long)

In [ ]:
def on_click_store(b):
    # This will take a very long time!
    all_the_structures = molecule_circus()
    i = 0
    # mol_name = struct_browser.results.value.description.split(' ')[0]
    mol_name = text_molname.value
    for n, atoms in all_the_structures.items():
        s = StructureData(ase=atoms)
        s.label = 'molecular circus'
        s.description = '{} ~ {}'.format(mol_name, n)
        i += 1
        print i
        s.store()
        print("Stored in AiiDA: "+repr(s))

text_molname = ipw.Text(description='Molecule name',
                        placeholder='ftX-abc')
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.VBox([text_molname, btn_store]))